# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [5]:
pd.set_option('max_columns', None)

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [6]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,Alley_missing,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MasVnrType_missing,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtQual_missing,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_TA,BsmtCond_missing,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_missing,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType1_missing,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtFinType2_missing,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,Electrical_missing,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,FireplaceQu_missing,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_missing,GarageFinish

In [7]:
X_train.shape

(1051, 296)

## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [8]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units

baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units

baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer

baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 116us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 48us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 33us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 33us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 33us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 34

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [12]:
# Train the mod# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 42us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 39us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [======

1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 34us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 34us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 34

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [13]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train) 

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [14]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size=32, epochs=150, validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.4364 - mean_squared_error: 0.4364 - val_loss: 0.1987 - val_mean_squared_error: 0.1987
Epoch 2/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.2274 - mean_squared_error: 0.2274 - val_loss: 0.1612 - val_mean_squared_error: 0.1612
Epoch 3/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1840 - mean_squared_error: 0.1840 - val_loss: 0.1591 - val_mean_squared_error: 0.1591
Epoch 4/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1640 - mean_squared_error: 0.1640 - val_loss: 0.1375 - val_mean_squared_error: 0.1375
Epoch 5/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.1477 - mean_squared_error: 0.1477 - val_loss: 0.1319 - val_mean_squared_error: 0.1319
Epoch 6/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.1331 - mean_squared_error: 0.1331 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1011 - val_mean_squared_error: 0.1011
Epoch 51/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0316 - mean_squared_error: 0.0316 - val_loss: 0.1024 - val_mean_squared_error: 0.1024
Epoch 52/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0309 - mean_squared_error: 0.0309 - val_loss: 0.1025 - val_mean_squared_error: 0.1025
Epoch 53/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0302 - mean_squared_error: 0.0302 - val_loss: 0.1025 - val_mean_squared_error: 0.1025
Epoch 54/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0291 - mean_squared_error: 0.0291 - val_loss: 0.1040 - val_mean_squared_error: 0.1040
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0286 - mean_squared_error: 0.0286 - val_loss: 0.1036 - val_mean_squared_error: 0.103

1051/1051 [==============================] - 0s 41us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1108 - val_mean_squared_error: 0.1108
Epoch 100/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1098 - val_mean_squared_error: 0.1098
Epoch 101/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1112 - val_mean_squared_error: 0.1112
Epoch 102/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 103/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1109 - val_mean_squared_error: 0.1109
Epoch 104/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0134 - mean_squared_error: 0.0134 - val_loss: 0.1116 - val_mean_squared_error: 0.1116
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.1156 - val_mean_squared_error: 0.1156
Epoch 149/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1154 - val_mean_squared_error: 0.1154
Epoch 150/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.1165 - val_mean_squared_error: 0.1165


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 22us/step


[0.007862055645396066, 0.007862055645396066]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [17]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 30us/step


[0.11645060256406835, 0.11645060256406835]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [19]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26816.517353756582

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, specify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [20]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 218us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1703 - val_mean_squared_error: 0.1703
Epoch 2/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.2217 - mean_squared_error: 0.2217 - val_loss: 0.1432 - val_mean_squared_error: 0.1432
Epoch 3/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.1793 - mean_squared_error: 0.1793 - val_loss: 0.1376 - val_mean_squared_error: 0.1376
Epoch 4/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.1568 - mean_squared_error: 0.1568 - val_loss: 0.1249 - val_mean_squared_error: 0.1249
Epoch 5/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.1404 - mean_squared_error: 0.1404 - val_loss: 0.1193 - val_mean_squared_error: 0.1193
Epoch 6/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.1263 - mean_squared_error: 0.1263 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0303 - mean_squared_error: 0.0303 - val_loss: 0.1089 - val_mean_squared_error: 0.1089
Epoch 51/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0297 - mean_squared_error: 0.0297 - val_loss: 0.1090 - val_mean_squared_error: 0.1090
Epoch 52/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1100 - val_mean_squared_error: 0.1100
Epoch 53/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 54/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 55/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0278 - mean_squared_error: 0.0278 - val_loss: 0.1100 - val_mean_squared_error: 0.110

1051/1051 [==============================] - 0s 52us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 100/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 101/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 102/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1185 - val_mean_squared_error: 0.1185
Epoch 103/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1187 - val_mean_squared_error: 0.1187
Epoch 104/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0135 - mean_squared_error: 0.0135 - val_loss: 0.1211 - val_mean_squared_error: 0.1211
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1254 - val_mean_squared_error: 0.1254
Epoch 149/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1249 - val_mean_squared_error: 0.1249
Epoch 150/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1253 - val_mean_squared_error: 0.1253


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 21us/step


[0.008126452382207341, 0.008126452382207341]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`) 

In [24]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 27us/step


[0.12534519954949278, 0.12534519954949278]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [26]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer

lecun_model.add(layers.Dense(100, activation='relu', kernel_initializer='lecun_normal', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 159us/step - loss: 0.4775 - mean_squared_error: 0.4775 - val_loss: 0.1466 - val_mean_squared_error: 0.1466
Epoch 2/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.2097 - mean_squared_error: 0.2097 - val_loss: 0.1232 - val_mean_squared_error: 0.1232
Epoch 3/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.1699 - mean_squared_error: 0.1699 - val_loss: 0.1171 - val_mean_squared_error: 0.1171
Epoch 4/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1088 - val_mean_squared_error: 0.1088
Epoch 5/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.1331 - mean_squared_error: 0.1331 - val_loss: 0.1059 - val_mean_squared_error: 0.1059
Epoch 6/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1198 - mean_squared_error: 0.1198 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0266 - mean_squared_error: 0.0266 - val_loss: 0.1163 - val_mean_squared_error: 0.1163
Epoch 51/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0261 - mean_squared_error: 0.0261 - val_loss: 0.1176 - val_mean_squared_error: 0.1176
Epoch 52/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0258 - mean_squared_error: 0.0258 - val_loss: 0.1189 - val_mean_squared_error: 0.1189
Epoch 53/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0252 - mean_squared_error: 0.0252 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 54/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.1193 - val_mean_squared_error: 0.1193
Epoch 55/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0243 - mean_squared_error: 0.0243 - val_loss: 0.1191 - val_mean_squared_error: 0.119

1051/1051 [==============================] - 0s 40us/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.1291 - val_mean_squared_error: 0.1291
Epoch 100/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1285 - val_mean_squared_error: 0.1285
Epoch 101/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1291 - val_mean_squared_error: 0.1291
Epoch 102/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1275 - val_mean_squared_error: 0.1275
Epoch 103/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1282 - val_mean_squared_error: 0.1282
Epoch 104/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.1303 - val_mean_squared_error: 0.1303
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1363 - val_mean_squared_error: 0.1363
Epoch 149/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1347 - val_mean_squared_error: 0.1347
Epoch 150/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0071 - mean_squared_error: 0.0071 - val_loss: 0.1360 - val_mean_squared_error: 0.1360


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 22us/step


[0.007057322658417652, 0.007057322658417652]

Evaluate the model (`lecun_model`) on validate data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 32us/step


[0.13601698652759706, 0.13601698652759706]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [29]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model

rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 177us/step - loss: 0.3205 - mean_squared_error: 0.3205 - val_loss: 0.1368 - val_mean_squared_error: 0.1368
Epoch 2/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1666 - mean_squared_error: 0.1666 - val_loss: 0.1115 - val_mean_squared_error: 0.1115
Epoch 3/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.1253 - mean_squared_error: 0.1253 - val_loss: 0.1250 - val_mean_squared_error: 0.1250
Epoch 4/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0979 - mean_squared_error: 0.0979 - val_loss: 0.1073 - val_mean_squared_error: 0.1073
Epoch 5/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0876 - mean_squared_error: 0.0876 - val_loss: 0.1002 - val_mean_squared_error: 0.1002
Epoch 6/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0659 - mean_squared_error: 0.0659 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0094 - mean_squared_error: 0.0094 - val_loss: 0.1019 - val_mean_squared_error: 0.1019
Epoch 51/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.0920 - val_mean_squared_error: 0.0920
Epoch 52/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0976 - val_mean_squared_error: 0.0976
Epoch 53/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0911 - val_mean_squared_error: 0.0911
Epoch 54/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.0956 - val_mean_squared_error: 0.0956
Epoch 55/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0064 - mean_squared_error: 0.0064 - val_loss: 0.0880 - val_mean_squared_error: 0.088

1051/1051 [==============================] - 0s 48us/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0980 - val_mean_squared_error: 0.0980
Epoch 100/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0048 - mean_squared_error: 0.0048 - val_loss: 0.0852 - val_mean_squared_error: 0.0852
Epoch 101/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0859 - val_mean_squared_error: 0.0859
Epoch 102/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0858 - val_mean_squared_error: 0.0858
Epoch 103/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0868 - val_mean_squared_error: 0.0868
Epoch 104/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0879 - val_mean_squared_error: 0.0879
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0899 - val_mean_squared_error: 0.0899
Epoch 149/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0794 - val_mean_squared_error: 0.0794
Epoch 150/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0878 - val_mean_squared_error: 0.0878


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 22us/step


[0.005820029039914187, 0.005820029039914187]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [36]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 29us/step


[0.0877587889877109, 0.0877587889877109]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [33]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model

adam_model.compile(optimizer='Adam',
                  loss='mse',
                  metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 323us/step - loss: 0.3813 - mean_squared_error: 0.3813 - val_loss: 0.1656 - val_mean_squared_error: 0.1656
Epoch 2/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1614 - mean_squared_error: 0.1614 - val_loss: 0.1320 - val_mean_squared_error: 0.1320
Epoch 3/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.1126 - mean_squared_error: 0.1126 - val_loss: 0.1043 - val_mean_squared_error: 0.1043
Epoch 4/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0863 - mean_squared_error: 0.0863 - val_loss: 0.1008 - val_mean_squared_error: 0.1008
Epoch 5/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0740 - mean_squared_error: 0.0740 - val_loss: 0.1009 - val_mean_squared_error: 0.1009
Epoch 6/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0565 - mean_squared_error: 0.0565 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.1038 - val_mean_squared_error: 0.1038
Epoch 51/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.1028 - val_mean_squared_error: 0.1028
Epoch 52/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.1024 - val_mean_squared_error: 0.1024
Epoch 53/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.1096 - val_mean_squared_error: 0.1096
Epoch 54/150
1051/1051 [==============================] - 0s 60us/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.0995 - val_mean_squared_error: 0.0995
Epoch 55/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0990 - val_mean_squared_error: 0.099

1051/1051 [==============================] - 0s 51us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0880 - val_mean_squared_error: 0.0880
Epoch 100/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0937 - val_mean_squared_error: 0.0937
Epoch 101/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0892 - val_mean_squared_error: 0.0892
Epoch 102/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0937 - val_mean_squared_error: 0.0937
Epoch 103/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0892 - val_mean_squared_error: 0.0892
Epoch 104/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0960 - val_mean_squared_error: 0.0960
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0897 - val_mean_squared_error: 0.0897
Epoch 149/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0048 - mean_squared_error: 0.0048 - val_loss: 0.0887 - val_mean_squared_error: 0.0887
Epoch 150/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0898 - val_mean_squared_error: 0.0898


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 24us/step


[0.003781442689513531, 0.003781442689513531]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [35]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 29us/step


[0.08982045256564598, 0.08982045256564598]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [37]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 33us/step


[0.14044084695920553, 0.14044084695920553]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [38]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29449.518219315825

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.